***TESTING THE FACE EMOTION RECOGNITION MODEL USING WEBCAM FEED***

---



In this part, we will test our model in real-time using face detection.

# 1.Importing the necessary libaries

In [2]:
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

# 2.Loading the saved model
Let’s start by loading the trained model architecture and weights so that it can be used further to make predictions.

In [3]:
# Load model from JSON file
json_file = open('fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

# Load weights and them to model
model.load_weights('fer.h5')

# 3.Loading Har-Cascade for Face Detection
We are using Haar-cascade for the detection position of faces and after getting position we will crop the faces.

haarcascade_frontalface_default can be downloaded using the link https://github.com/opencv/opencv/tree/master/data/haarcascades.

In [4]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 4.Read Frames and apply Preprocessing using OpenCV
Use OpenCV to read frames and for image processing.

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    if not ret:
        break

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.1, 6, minSize=(150, 150))

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)
        roi_gray = gray_img[y:y + w, x:x + h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255.0

        predictions = model.predict(img_pixels)
        max_index = int(np.argmax(predictions))

        emotions = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear']
        predicted_emotion = emotions[max_index]

* **predicted_emotion** returns the label of emotion.

* Normalize test images by dividing them by 255.

* **np.expand_dims** convert a 3D matrix into a 4D tensor.

* **(x,y,w,h)** are the coordinates of faces in the input frame.
haar_cascade takes only grayscale images.

# 5.Adding Overlay
Adding an overlay on the output frame and displaying the prediction with confidence gives a better look.

In [5]:
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    if not ret:
        break

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.1, 6, minSize=(150, 150))

    for (x, y, w, h) in faces_detected:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)
        roi_gray = gray_img[y:y + w, x:x + h]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis=0)
        img_pixels /= 255.0

        predictions = model.predict(img_pixels)
        max_index = int(np.argmax(predictions))

        emotions = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear']
        predicted_emotion = emotions[max_index]

        cv2.putText(img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

        resized_img = cv2.resize(img, (1000, 700))
        cv2.imshow('Facial Emotion Recognition', resized_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Now run it !!!

# 6.Conclusion
We have seen how to preprocess data, design a network that is capable of classifying the emotions, and then use Opencv
for the detection of the faces and then pass it for prediction.